In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import math
import bs4 as bs
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Import for text analytics
import spacy
from spacy import displacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess
from gensim import corpora
import multiprocessing

# Load English language model of spacy
sp = spacy.load('en_core_web_sm')

# Sklearn import
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score, recall_score, f1_score 
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Import OpenAI
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
!pip install openai
import openai

# Other imports
!pip install PyPDF2
!pip install PyCryptodome
import nltk
nltk.download('punkt')
import PyPDF2
from PyPDF2 import PdfWriter, PdfReader
import re
import requests
from bs4 import BeautifulSoup
import torch
!pip install transformers
import transformers as ppb
import warnings
warnings.filterwarnings('ignore') 

[nltk_data] Downloading package punkt to /Users/guillaume/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



# Introduction to the Notebook

The primary purposes of this notebook are:

- Extract text from different banks annual report 
- Applying our best and less expensive model to predict envirenmental claims in these reports 
- In conjunction with Trucost's climate data, an analysis of these annual reports is being developed

The selected banks for analysis include:

- UBS
- Credit Suisse
- Banque Cantonale Vaudoise
- J.P. Morgan
- Goldman Sachs

We examined the annual reports of each bank for the years 2019 and 2020. Therefore, we use our prediction model a total of 10 times.

Furthermore, we identified specific sections within these reports that were pertinent to our analysis and likely to contain environmental claims. For instance, we excluded the "Financial Statements" sections from all the annual reports. This process was conducted individually for each report, carefully determining which sections were relevant to our analysis.

The notebook is organized as follows:
- To begin, we leverage our most effective and less expensive text classifier, the GPT-3 utilizing the ada model, to develop a function for fine-tuning each sentence within the annual reports. This function will enables us to predict whether a sentence from a bank's annual report qualifies as an environmental claim or not. We did not want to use the very best text classifier GPT-3 utilizing the davinci model for financial reasons.
- Simultaneously, we extract relevant sections of the annual reports for each bank in the years 2019 and 2020. These sections are stored in dedicated dataframes specific to each bank and year. Subsequently, these dataframes are incorporated into the aforementioned GPT-3 function.
- Towards the end of the notebook, two analyses are conducted. The first analysis focuses on individual bank CO2 emissions, comparing the emissions between different years. The second analysis centers on an industry-wide comparison, facilitated by the inclusion of the CO2 intensity metric.

# OpenAi - Prediction Function

First we "import" our OpenAi Key. For confidentiality reasons, we cannot disclose the key within this notebook

In [ ]:
# OpenAi key
api_key = '***'
openai.api_key = api_key

In the next cell, we define a function that takes the sentences extracted from the annual reports of banks as input. The function utilizes the GPT-3 Text Classifier to make predictions and generates a dataframe containing the established predictions. 

In [ ]:
def openai_predictions_ada (dataframe):
    #Transforming the dataset to fit openai's formatting
    dataframe = dataframe.rename(columns={'Sentences':'prompt'})
    dataframe['prompt'] = dataframe['prompt'] + '->'

    # Make predictions for each sentence
    predictions = []
    for index, row in dataframe.iterrows():
        new_prompt = row["prompt"]
        answer = openai.Completion.create(
          model="ada:ft-personal-2023-05-20-17-34-23",
          prompt=new_prompt,
          max_tokens=2,  # Adjust based on your desired prediction length
          temperature=0,
          top_p = 0.1
      )
        difficulty = answer.choices[0]['text']
        predictions.append(difficulty)

    # Add the predictions to the dataframe
    dataframe["pred"] = predictions
    #clean the prediction column
    dataframe['pred'] = dataframe['pred'].str.replace('.', '')

    #display the dataframe once the predictions are done
    display(dataframe)
    display(dataframe.groupby('pred').count())

    return dataframe

# Banks Annual Reports - Predictions 

In this section, we conduct a thorough examination of the five banks individually for the two years being reviewed (2019 and 2020). For each bank, the process is the same. The process begins by extracting the text from each bank's annual report. Subsequently, we use the function we developed earlier to analyze and predict the content of each sentence within the extracted text. Next, we store the count of environmental claims per report in a bank- and year-specific variable.  This particular variable will be further used in our analysis that incorporates Trucost's climate data.

## Banque cantonal Vaudoise

The first bank to be investigated is the Banque Cantonale Vaudoise.

### 2020 Annual Report

As an example, in the BCV 2020 annual report, we stop at page 76 since the subsequent sections focus on the governance structure and financial statements. It's important to note that the code's numbering does not align directly with the page numbers in the bottom right corner of the annual report. This discrepancy arises because the annual report excludes introductory pages when counting pages, unlike the "pdf" version. This variation persists across all the analyzed reports in this notebook.

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/ENG-BCV_Rapport_annuel_2020.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("BCV_annual_report_2020.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
    
# Open the input PDF file
infile = PdfReader("BCV_annual_report_2020.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 76): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("BCV_annual_report_2020_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("BCV_annual_report_2020_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence --> Formating of the setences 
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]

# Create a DataFrame of the extracted sentences
df_bcv_20 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_bcv_20 = openai_predictions_ada(df_bcv_20)

,prompt,pred
0,2020 Annual ReportHead Office Place St-Françoi...,0
1,BCV at a glance 2020 was a great year for our ...,0
2,"• On 28 May 2020, we carried out a 10-for-1 st...",0
3,"/T_he following day, our share was added to th...",0
4,We continued to implement our sustainable deve...,0
...,...,...
1181,/T_his chapter explains how the Bank puts thes...,0
1182,six-exchange-regulation.com/dam/downloads/regu...,0
1183,Group structure and shareholders 75 2.->,0
1184,"Compensation, shareholdings, and loans 96 6.->",0


,prompt
pred,
0,1152
1,34


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_bcv_20.to_csv('df_bcv_20_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
bcv_20_env_claims = df_bcv_20[df_bcv_20['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_bcv_20[(df_bcv_20['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_bcv_20[(df_bcv_20['pred']==' 0')].iloc[1][0])

Green claim:  It oﬀers a broad, transparent view of what we are doing to fulﬁll our commitment to promoting economically, socially, and environmentally sustainable development.-> 

Non-green claim:  BCV at a glance 2020 was a great year for our share • Our share delivered a total return of 26.5% in 2020,  making it the second-best performer among Swiss  banking stocks.->


### 2019 Annual Report

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/ENG-BCV_Rapport_annuel_2019.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("BCV_annual_report_2019.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("BCV_annual_report_2019.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 72): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("BCV_annual_report_2019_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("BCV_annual_report_2019_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_bcv_19 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_bcv_19 = openai_predictions_ada(df_bcv_19)

,prompt,pred
0,BCV at a glance • We maintained or slightly ad...,0
1,• The Board of Directors is recommending that ...,0
2,The UNPRI are the world’s leading framework fo...,0
3,"• Operating profit increased 4% to CHF 419m, ...",0
4,• Net profit was up 4% to CHF 363m.->,0
...,...,...
989,This chapter explains how the Bank puts these ...,0
990,six-exchange-regulation.com/dam/downloads/regu...,0
991,Group structure and shareholders 71 2.->,0
992,"Compensation, shareholdings, and loans 92 6.->",0


,prompt
pred,
0,978
1,16


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_bcv_19.to_csv('df_bcv_19_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
bcv_19_env_claims = df_bcv_19[df_bcv_19['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_bcv_19[(df_bcv_19['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_bcv_19[(df_bcv_19['pred']==' 0')].iloc[1][0])

Green claim:  Beyond these events, we see this anniversary as an  opportunity to highlight BCV’s core guiding principles: close  ties with the community we serve and a commitment to  contributing to our home region by fostering economically,  environmentally, and socially sustainable development.-> 

Non-green claim:  • The Board of Directors is recommending that shareholders approve a 10-for-1 stock split, in order to  enhance the liquidity of BCV’s share.->


## UBS

The second bank to be investigated is UBS.

### 2020 Annual Report

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/full-report-ubs-group-ag-consolidated-2020-en.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("UBS_annual_report_2020.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
    
# Open the input PDF file
infile = PdfReader("UBS_annual_report_2020.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 73) or (143 <= i <= 144): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("UBS_annual_report_2020_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("UBS_annual_report_2020_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words. 
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_ubs_20 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_ubs_20 = openai_predictions_ada(df_ubs_20)

,prompt,pred
0,UBS Group AG Annual Report 2020->,0
1,We also provide a combined annual report for ...,0
2,"We provide our combined Annual Report, the Pil...",0
3,"The reports are presented in US dollars, our p...",0
4,The UBS Group AG Annual Report 2020 is partly ...,0
...,...,...
2035,Cross-border risk remains an area of regulator...,0
2036,There is also ongoing high attention on the ri...,0
2037,UBS actively assesses and applies permanent e...,0
2038,"During 2020, thanks to the continued focus on ...",0


,prompt
pred,
0,1955
1,85


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_ubs_20.to_csv('df_ubs_20_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
ubs_20_env_claims = df_ubs_20[df_ubs_20['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_ubs_20[(df_ubs_20['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_ubs_20[(df_ubs_20['pred']==' 0')].iloc[1][0])

Green claim:  We delivered the best of UBS to our clients and  extended our leadership in sustainability.-> 

Non-green claim:  We also provide a combined annual report for  UBS Group AG and UBS AG consolidated, which additionally  includes the consolidated financial statements of UBS AG as  well as supplemental disclosures required under SEC  regulations and is the basis for our SEC Form 20-F filing.->


### 2019 Annual Report

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/full-report-ubs-group-ag-consolidated-2019-en.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("UBS_annual_report_2019.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("UBS_annual_report_2019.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 77) or (i == 118) or (i == 113): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("UBS_annual_report_2019_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("UBS_annual_report_2019_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words.
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_ubs_19 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_ubs_19 = openai_predictions_ada(df_ubs_19)

,prompt,pred
0,UBS Group AG Annual Report 2019->,0
1,Annual reporting At the center of our external...,0
2,We also provide a combined annual report for U...,0
3,"The reports are presented in US dollars, our p...",0
4,The UBS Group AG Annual Report 2019 is transla...,0
...,...,...
1453,Heightened regulatory expectations and attenti...,0
1454,Refer to “Operational risk” in this section an...,0
1455,We make use of both scenario-based stress test...,0
1456,These complementary frameworks SWCPVKVCV...,0


,prompt
pred,
0,1392
1,66


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_ubs_19.to_csv('df_ubs_19_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
ubs_19_env_claims =  df_ubs_19[df_ubs_19['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_ubs_19[(df_ubs_19['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_ubs_19[(df_ubs_19['pred']==' 0')].iloc[1][0])

Green claim:  Society and environment  • USD 488.5 billion of sustainable investing assets (13.5% of our total invested assets).-> 

Non-green claim:  Annual reporting At the center of our external reporting approach is the  annual report of UBS Group AG, which consists of disclosures for UBS Group AG and its consolidated subsidiaries.->


## Crédit Suisse 

The third bank to be investigated is Crédit Suisse.

### 2020 Annual Report

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/csg-ar-2020-en.pdf"
# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("CS_annual_report_2020.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("CS_annual_report_2020.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 60) or (141 <= i <= 181): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("CS_annual_report_2020_trimmed.pdf", "wb") as f:
    output.write(f)
    
    
# Load PDF file
pdf = PdfReader("CS_annual_report_2020_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_cs_20 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_cs_20 = openai_predictions_ada(df_cs_20)

,prompt,pred
0,Credit Suisse Group AG Annual Report 2020->,0
1,Annual Report 2020 Credit Suisse Group AG->,0
2,"For the purposes of this report, unless the c...",0
3,"The business of Credit Suisse AG, the direct b...",0
4,We use the term the “Bank” when we are referri...,0
...,...,...
2163,Representations and warranties on residential ...,0
2164,We have provided these representations and war...,0
2165,The loans sold are primarily loans that we hav...,0
2166,If it is determined that rep - resentations an...,0


,prompt
pred,
0,2139
1,29


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_cs_20.to_csv('df_cs_20_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
cs_20_env_claims = df_cs_20[df_cs_20['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_cs_20[(df_cs_20['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_cs_20[(df_cs_20['pred']==' 0')].iloc[30][0])

Green claim:  We launched a new Sustainability, Research & Investment Solutions (SRI) function at the Executive Board level, underlining the sharpened focus on sustainability.-> 

Non-green claim:  This was  a decline of 22% year on year including primarily the effects of  higher provision for credit losses and major litigation provisions, as well as the impairment to the valuation of our minority sharehold - ing in York Capital Management.->


### 2019 Annual Report

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/csg-ar-2019-en.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("CS_annual_report_2019.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("CS_annual_report_2019.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 55) or (137 <= i <= 159): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("CS_annual_report_2019_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("CS_annual_report_2019_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_cs_19 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_cs_19 = openai_predictions_ada(df_cs_19)

,prompt,pred
0,Credit Suisse Group AG Annual Report 2019->,0
1,Annual Report 2019 Credit Suisse Group AG->,0
2,"For the purposes of this report, unless the c...",0
3,"The business of Credit Suisse AG, the direct b...",0
4,We use the term the “Bank” when we are referri...,0
...,...,...
1749,Governance of fiduciary riskSound governance i...,0
1750,"Our program targets daily, monthly or quarterl...",0
1751,Formal review meetings are in place to ensure...,0
1752,Pension risk Definition and sources of pension...,0


,prompt
pred,
0,1733
1,21


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_cs_19.to_csv('df_cs_19_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
cs_19_env_claims = df_cs_19[df_cs_19['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_cs_19[(df_cs_19['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_cs_19[(df_cs_19['pred']==' 0')].iloc[30][0])

Green claim:  Second, we are focusing  on delivering sustainable finance solutions that help our clients  achieve their goals and contribute to the realization of the UN Sustainable Development Goals (SDGs).-> 

Non-green claim:  According to the Credit Suisse Global Wealth Report  2019, the global pool of wealth grew once again between mid-2018 and mid-2019, increasing by 2.6%.->


## J.P. Morgan

The fourth bank to be investigated is J.P. Morgan.

### Annual Report 2020

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/JPM_annualreport-2020.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("JPM_annual_report_2020.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("JPM_annual_report_2020.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 97): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("JPM_annual_report_2020_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("JPM_annual_report_2020_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_jpm_20 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_jpm_20 = openai_predictions_ada(df_jpm_20)

,prompt,pred
0,more than loans funded280K to advance racial ...,0
1,Refer to Explanation and Reconciliation of the...,0
2,(b) Refer to Liquidity Risk Management on pa...,0
3,(c) The ratios presented are calculated unde...,0
4,Refer to Capital Risk Management on pages 91–1...,0
...,...,...
1981,"As of February 19 , 2021, the Firm has funded...",0
1982,"Beginning in March 2020, the Federal Reserve ...",0
1983,The Firm has participated and is participating...,0
1984,Refer to Capital Risk Management on pages 91-...,0


,prompt
pred,
0,1964
1,22


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_jpm_20.to_csv('df_jpm_20_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
jpm_20_env_claims = df_jpm_20[df_jpm_20['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_jpm_20[(df_jpm_20['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_jpm_20[(df_jpm_20['pred']==' 0')].iloc[18][0])

Green claim:  We respect the people in  our communities and protect the environment by embracing sustainable  practices across our businesses.-> 

Non-green claim:  As you know, we have long cham - pioned the essential role of banking in a community — its potential for bringing  people together, for enabling companies and individuals to reach for their dreams,->


### Annual Report 2019

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/JPM_annualreport-2019.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("JPM_annual_report_2019.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("JPM_annual_report_2019.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 51): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("JPM_annual_report_2019_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("JPM_annual_report_2019_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_jpm_19 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_jpm_19 = openai_predictions_ada(df_jpm_19)

,prompt,pred
0,Refer to Explanation and Reconciliation of the...,0
1,(b) Refer to Liquidity Risk Management on pa...,0
2,(c) The ratios presented are calculated unde...,0
3,Refer to Capital Risk Management on pages 85-9...,0
4,JPMorgan Chase & Co. (NYSE: JPM) is a leading ...,0
...,...,...
796,The U.S . Treasur y Depar tmen t has issued pr...,0
797,"Under the pr oposed r egulations, amendmen ts ...",0
798,The F irm c ontinues to monitor the tr ansitio...,0
799,The F irm also c ontinues to de velop and impl...,0


,prompt
pred,
0,791
1,10


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_jpm_19.to_csv('df_jpm_19_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
jpm_19_env_claims = df_jpm_19[df_jpm_19['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_jpm_19[(df_jpm_19['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_jpm_19[(df_jpm_19['pred']==' 0')].iloc[19][0])

Green claim:  Over the last five years, for example,  we’ve used technology and machine learning to reduce fraud losses in the credit card business by 50%.-> 

Non-green claim:  Looking back on the last two decades — starting from my time as CEO of Bank  One in 2000 — the firm has weathered some unprecedented challenges, as we  will with this current pandemic, but they did not stop us from accomplishing  some extraordinary things.->


## Goldman Sachs
The fifth bank to be investigated is Goldman Sachs.

### Annual Report 2020

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/GS_annual-report-2020.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("GS_annual_report_2020.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("GS_annual_report_2020.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 67): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("GS_annual_report_2020_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("GS_annual_report_2020_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_gs_20 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can utilize the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_gs_20 = openai_predictions_ada(df_gs_20)

,prompt,pred
0,Goldman Sachs 2020 Annual Report 1 Last year ...,0
1,"Throughout my more-than-35-year career, ther...",0
2,The contrast between the hardship imposed on a...,0
3,"The pandemic put enormous strain on everyone, ...",0
4,And though I was pleased to see central banks ...,0
...,...,...
1184,"In the preceding paragraphs, square footage fi...",0
1185,We regularly evaluate our space capacityin rel...,0
1186,We mayincur exit costs in the future if we (i)...,0
1187,These costs may bematerial to our operating re...,0


,prompt
pred,
0,1181
1,8


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_gs_20.to_csv('df_gs_20_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
gs_20_env_claims = df_gs_20[df_gs_20['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_gs_20[(df_gs_20['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_gs_20[(df_gs_20['pred']==' 0')].iloc[1][0])

Green claim:  For example, we have launched partner-led  sustainability councils within each of our divisions.-> 

Non-green claim:  Throughout my more-than-35-year   career, there have always been disruptions, but this year saw a healthcare crisis that had   a personal impact on millions.->


### Annual Report 2019

In [ ]:
# Set the URL of the PDF file
url = "https://github.com/noelopez-E4S/class_datascience/raw/main/GS_annual-report-2019.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the PDF file content from the response content
    pdf_content = response.content
    # Save the PDF file to disk
    with open("GS_annual_report_2019.pdf", "wb") as f:
        f.write(pdf_content)
    print("PDF file downloaded successfully!")
else:
    print(f"Failed to download the PDF file. Error code: {response.status_code}")
    
from PyPDF2 import PdfWriter, PdfReader

# Open the input PDF file
infile = PdfReader("GS_annual_report_2019.pdf", "rb")

# Create a new PDF writer
output = PdfWriter()

# Loop through all pages and add them to the new PDF writer
for i, page in enumerate(infile.pages):
    if (i < 45): # define which pages of the report the model has to extract --> only the relevant pages 
        output.add_page(page)

# Write the output PDF to a file
with open("GS_annual_report_2019_trimmed.pdf", "wb") as f:
    output.write(f)
    
# Load PDF file
pdf = PdfReader("GS_annual_report_2019_trimmed.pdf", "rb")

# Loop through all pages and extract each sentence
sentences = []
for page in pdf.pages:
    text = page.extract_text()
    page_sentences = nltk.tokenize.sent_tokenize(text)
    sentences.extend(page_sentences)

# Replace all '\n' with spaces in each sentence
sentences = [re.sub(r"\n", " ", s) for s in sentences]

# Filter out all sentences with less than 5 words
sentences = [s for s in sentences if  5 < len(s.split()) < 50]


# Create a DataFrame of the extracted sentences
df_gs_19 = pd.DataFrame(sentences, columns=["Sentences"])

PDF file downloaded successfully!


With all the sentences from the relevant sections of the annual reports now compiled in a dataframe, we can use the previously established function to predict each sentence.

In [ ]:
# Calling the prediction function
df_gs_19 = openai_predictions_ada(df_gs_19)

,prompt,pred
0,Goldman Sachs 2019 Annual ReportThe Goldman Sa...,0
1,This includes pressure on the nonprofit secto...,0
2,Government action generally has been swift an...,0
3,This fluid and historic situation is having a...,0
4,"As a firm, we are taking actions to support ou...",0
...,...,...
817,New and prospective liquidity-relatedregulatio...,0
818,Given the overlap and complexinteractions amon...,0
819,We face enhanced risks as new business initiat...,0
820,A number of our recent and planned business in...,0


,prompt
pred,
0,820
1,2


In [ ]:
# Storing the dataframe with predtictions --> to avoid re-running the model everytime
df_gs_19.to_csv('df_gs_19_ada.csv', index=False)

Now we can count the number of environmental claims (label = 1) predicted by our model. This variable will be used as part of our analysis with Trucost data at the end of the notebook.

In [ ]:
# Count the number of observation with 1 as a label
gs_19_env_claims = df_gs_19[df_gs_19['pred']==' 1'].count()[1]

You can find below an example of environmental and non-environmental claims:

In [ ]:
#Green claim
print('Green claim: ',df_gs_19[(df_gs_19['pred']==' 1')].iloc[1][0],'\n')

#Non-green claim
print('Non-green claim: ',df_gs_19[(df_gs_19['pred']==' 0')].iloc[1][0])

Green claim:  We are  focused on ensuring our efforts in this area are aligned  with and accretive to our overall sustainability objectives.-> 

Non-green claim:  This includes pressure  on the nonprofit sector, which provides critical services to the most vulnerable.->


# Banks Annual Reports - Comprehensive analysis 

In this section, we will perform an analysis by integrating the predictions of our previously conducted model in the notebook with the climatic data obtained from Trucost. These climate data include the climate performance of all the banks investigated in our study. Our objective is to compare this climate data with the number of environmental claims identified by our GPT-3 Text Classifier within the 10 annual reports.

First, we import the Trucost data, was provided by Dr. Boris Thurm.

In [ ]:
# Import Trucot data
data = pd.read_csv("companies_environmental-performance.csv")

### First analysis: Absolute YoY comparison

During the initial analysis, our focus will be on examining each selected bank individually. We will compare the variations in their number of environmental claims between two consecutive years with the corresponding changes in their CO2 emissions over the same period. The objective of this comparative assessment is to determine whether there is alignment or disparity between the bank's increased emphasis on climate change through environmental claims in their annual reports and the actual trajectory of their CO2 emissions. If we observe a noteworthy increase in environmental claims accompanied by a simultaneous rise in CO2 emissions, it could raise concerns regarding possible greenwashing practices.

In the next cell, a function is established to retrieve form trucost climate data for a specific bank and a particular year.

In [ ]:
def calculate_carbon_footprint(company_name, data, year):
    """
    Calculates the sum of 'Carbon-Scope 1 (tonnes CO2e)', 'Carbon-Scope 2 (tonnes CO2e)',
    and 'Carbon-Scope 3 (tonnes CO2e)' for a given company and year.
    
    Args:
        company_name (str): The name of the company to calculate the carbon footprint for.
        data (pandas.DataFrame): The DataFrame containing the environmental performance data.
        year (int): The year for which to calculate the carbon footprint. Default is 2020.
        
    Returns:
        float: The sum of 'Carbon-Scope 1 (tonnes CO2e)', 'Carbon-Scope 2 (tonnes CO2e)',
            and 'Carbon-Scope 3 (tonnes CO2e)' for the given company and year.
    """
    
    # Filter the DataFrame to get rows for the given company and year, and columns for the carbon scopes
    company_data = data[(data['Company'] == company_name) & (data['Financial Year'] == year)][['Carbon-Scope 1  (tonnes CO2e)',
                                                                                              'Carbon-Scope 2  (tonnes CO2e)',
                                                                                              'Carbon-Scope 3 (tonnes CO2e)']]
    
    # Check that the company has a non-null value in each of the carbon scope columns
    if company_data.isnull().values.any():
        raise ValueError("Company has a missing value in one or more carbon scope columns")
    
    # Calculate the sum of the carbon scope columns for the given company and year
    carbon_footprint = company_data.sum().sum()
    
    return carbon_footprint

Now we can use the function to retrieve the relevant data for the analyzed banks and the specified years.

Note: In the case of UBS for the year 2019, we had to manually import the data. Please refer to the next box for detailed explanations

In [ ]:
# Calling the corresponding function with the comapany name and the year 
ubs_20_co2 = calculate_carbon_footprint("UBS Group AG", data, 2020)
ubs_19_co2 = (11320 + 142636 + 576277) # see comment below
cs_20_co2 = calculate_carbon_footprint("Credit Suisse Group AG", data, 2020)
cs_19_co2 = calculate_carbon_footprint("Credit Suisse Group AG", data, 2019)
bcv_20_co2 = calculate_carbon_footprint("Banque Cantonale Vaudoise", data, 2020)
bcv_19_co2 = calculate_carbon_footprint("Banque Cantonale Vaudoise", data, 2019)
jpm_20_co2 = calculate_carbon_footprint("JPMorgan Chase & Co.", data, 2020)
jpm_19_co2 = calculate_carbon_footprint("JPMorgan Chase & Co.", data, 2019)
gs_20_co2 = calculate_carbon_footprint("The Goldman Sachs Group, Inc.", data, 2020)
gs_19_co2 = calculate_carbon_footprint("The Goldman Sachs Group, Inc.", data, 2019)

The dataset provided does not include the CO2 emissions for UBS Group for 2019. However, through the Sustainable Finance course taught by Eric Jondeau, we were provided with a Trucost Dataset that includes the CO2 emissions for UBS Group in 2019. To ensure the accuracy of this data, we compared it to the data from the provided dataset for the year 2018 and found that they were identical (see code in following cells). Therefore, we feel confident in incorporating the CO2 emissions data for UBS Group in 2019 from the other Trucost dataset. We remain at disposal to provide the corresponding file upon request.

In [ ]:
# check accuracy file provided by Prof. Jondeau for 2018
trucost_ubs_18_jondeau = 11521.9961 + 150957 + 843339.1029 # --> manual imported data from Prof. Jondeau dataset
print(trucost_ubs_18_jondeau)

# from the dataset provided in ML
ubs_18_co2 = calculate_carbon_footprint("UBS Group AG", data, 2018)
print(ubs_18_co2)

1005818.099
1005818.099


Now that the accuracy is verified, we have all the measures in hand. We can now present them in a table.

In [ ]:
# Define the display format for float values
pd.options.display.float_format = '{:.2f}'.format

# Create a dictionary with the data
new_data = {
    'Company': ['UBS', 'UBS', 'Credit Suisse', 'Credit Suisse', 'Banque Cantonale Vaudoise', 'Banque Cantonale Vaudoise', "JPMorgan Chase & Co.", "JPMorgan Chase & Co.", "The Goldman Sachs Group, Inc.", "The Goldman Sachs Group, Inc."],
    'Year': [2019, 2020, 2019, 2020, 2019, 2020, 2019, 2020, 2019, 2020],
    'Environmental Claims': [ubs_19_env_claims, ubs_20_env_claims, cs_19_env_claims, cs_20_env_claims, bcv_19_env_claims, bcv_20_env_claims, jpm_19_env_claims, jpm_20_env_claims, gs_19_env_claims, gs_20_env_claims],
    'CO2 Emissions': [ubs_19_co2, ubs_20_co2, cs_19_co2, cs_20_co2, bcv_19_co2, bcv_20_co2, jpm_19_co2, jpm_20_co2, gs_19_co2, gs_20_co2]
}

# Create a pandas dataframe
df = pd.DataFrame(new_data)

# Display the dataframe
df


,Company,Year,Environmental Claims,CO2 Emissions
0,UBS,2019,66,730233.00
1,UBS,2020,85,1065783.41
2,Credit Suisse,2019,21,735218.33
3,Credit Suisse,2020,29,678125.10
4,Banque Cantonale Vaudoise,2019,16,27463.08
5,Banque Cantonale Vaudoise,2020,34,29505.24
6,JPMorgan Chase & Co.,2019,10,3875536.24
7,JPMorgan Chase & Co.,2020,22,3479547.33
8,"The Goldman Sachs Group, Inc.",2019,2,1313970.93
9,"The Goldman Sachs Group, Inc.",2020,8,1327399.42


For the analysis this table, we kindly refer the reader of this notebook to the accompanying README file, which can be found on the GitHub repository.

In [ ]:
# Store the table 
df.to_csv('company_claims_emissions')

### Second analysis: Industry ranking

In this second analysis, we will expand our examination to the industry level, considering the diverse sizes of banks to account for differences in the amount emissions. To accomplish this, we will introduce a metric called CO2 intensity for each bank, which normalizes individual CO2 emissions based on the company's revenues. This normalization helps mitigate the influence of size discrepancies and enables meaningful comparisons across different companies within the same industry. By using this metric, we will establish a ranking of banks according to their CO2 intensity and compare it with the ranking based on the number of environmental claims. The purpose of this comparison is to identify cases where a bank, despite having a low CO2 intensity rank, extensively emphasizes its climate ambitions. Such disparities could potentially indicate instances of greenwashing practices. This analysis will span a two-year period as well.

In [ ]:
# Create a sub-data set with companies only active in the "Financials" sector
financial_companies = data[data['GICS Sector Name'] == 'Financials']

Now, we create a new function that will calculates the cumulative carbon intensity for a given company and year.

In [ ]:
def calculate_total_carbon_intensity(company_name, year):
    """
    Calculates the total carbon intensity for a given company and year.
    
    Args:
        company_name (str): The name of the company to calculate the total carbon intensity for.
        data (pandas.DataFrame): The DataFrame containing the environmental performance data.
        year (int): The year for which to calculate the total carbon intensity.
        
    Returns:
        float: The sum of carbon intensity for the given company and year.
    """
    
    # Filter the DataFrame to get rows for the given company and year, and select the specified columns
    company_data = financial_companies[(financial_companies['Company'] == company_name) & (data['Financial Year'] == year)][['Carbon Intensity-Scope 1 (tonnes CO2e/USD mn)',
                                                                                               'Carbon Intensity-Scope 2 (tonnes CO2e/USD mn)',
                                                                                               'Carbon Intensity-Scope 3 (tonnes CO2e/USD mn)']]
    
    # Check if the filtered DataFrame is empty
    if company_data.empty:
        raise ValueError("No data found for the specified company and year.")
    
    # Calculate the sum of the specified columns for the given company and year
    total_carbon_intensity = company_data.sum().sum()
    
    return total_carbon_intensity

Now we can use the function to retrieve the relevant data for the analyzed banks and the specified years.

Note: Similar to previous instances, we have an issue with UBS for the year 2019. Consequently, we used the data provided by Prof. Jondeau, which also included metrics related to CO2 intensity. We also perform an accuraycy check between the two files for the year of 2018.

In [ ]:
# Calling the corresponding function with the comapany name and the year 
jpm_20_intensity = calculate_total_carbon_intensity('JPMorgan Chase & Co.', 2020)
jpm_19_intensity = calculate_total_carbon_intensity('JPMorgan Chase & Co.', 2019)
gs_19_intensity = calculate_total_carbon_intensity('The Goldman Sachs Group, Inc.', 2020)
gs_20_intensity = calculate_total_carbon_intensity('The Goldman Sachs Group, Inc.', 2019)
ubs_20_intensity = calculate_total_carbon_intensity('UBS Group AG', 2020)
ubs_19_intensity = 0.5851 + 7.3726 + 29.7684 # check for accuracy bellow
bcv_20_intensity = calculate_total_carbon_intensity('Banque Cantonale Vaudoise', 2020)
bcv_19_intensity = calculate_total_carbon_intensity('Banque Cantonale Vaudoise', 2019)
cs_19_intensity = calculate_total_carbon_intensity('Credit Suisse Group AG', 2019)
cs_20_intensity = calculate_total_carbon_intensity('Credit Suisse Group AG', 2020)

In [ ]:
# check accuracy file provided by Prof. Jondeau for 2018
trucost_ubs_18_intensity_jondeau = 0.428018116901008 + 5.60773761006559+ 31.3282882237318 # --> manual imported data from Prof. Jondeau dataset
print(trucost_ubs_18_intensity_jondeau)

# from the dataset provided in ML
ubs_18_co2_intensity = ubs_20_intensity = calculate_total_carbon_intensity('UBS Group AG', 2018)
print(ubs_18_co2_intensity)

37.364043950698395
37.364043947


Now that the accuracy is also verified for carbon intensity, we have all the measures in hand. We can now present them in a table.

In [ ]:
# Create a dictionary with the given variables and their corresponding values
data = {
    'Year': [2020, 2019, 2020, 2019, 2020, 2019, 2020, 2019, 2020, 2019],
    'Name': ['JPMorgan Chase', 'JPMorgan Chase', 'Goldman Sachs', 'Goldman Sachs', 'UBS', 'UBS', 'BCV', 'BCV', 'Credit Suisse', 'Credit Suisse'],
    'CO2 Intensity': [jpm_20_intensity, jpm_19_intensity, gs_20_intensity, gs_19_intensity, ubs_20_intensity, ubs_19_intensity, bcv_20_intensity, bcv_19_intensity, cs_20_intensity, cs_19_intensity],
    'Environmental Claims': [jpm_20_env_claims, jpm_19_env_claims, gs_20_env_claims, gs_19_env_claims, ubs_20_env_claims, ubs_19_env_claims, bcv_20_env_claims, bcv_19_env_claims, cs_20_env_claims, cs_19_env_claims]

}
# Create a DataFrame from the data dictionary
df = pd.DataFrame(data)

# Sort the DataFrame by 'CO2 Intensity' in ascending order
df_sorted = df.sort_values(by='CO2 Intensity', ascending=True)

# Add a 'Rank - CO2 Intensity' column based on the sorted order
df_sorted['Rank - CO2 Intensity'] = range(1, len(df_sorted) + 1)

# Reorder the columns
df_sorted = df_sorted[['Year', 'Name', 'CO2 Intensity', 'Rank - CO2 Intensity', 'Environmental Claims']]

# Reset the index of the DataFrame
df_sorted = df_sorted.reset_index(drop=True)

# Sort the DataFrame by 'Environmental Claims' in descending order
df_sorted = df_sorted.sort_values(by='Environmental Claims', ascending=False)

# Add a 'Rank - Env. Claims' column based on the sorted order
df_sorted['Rank - Env. Claims'] = range(1, len(df_sorted) + 1)

# Reorder the columns
df_sorted = df_sorted[['Year', 'Name', 'CO2 Intensity', 'Rank - CO2 Intensity', 'Environmental Claims', 'Rank - Env. Claims']]

# Reset the index of the DataFrame
df_sorted = df_sorted.reset_index(drop=True)

# Print the sorted and ranked DataFrame
df_sorted

,Year,Name,CO2 Intensity,Rank - CO2 Intensity,Environmental Claims,Rank - Env. Claims
0,2020,UBS,37.36,9,85,1
1,2019,UBS,37.73,10,66,2
2,2020,BCV,29.31,1,34,3
3,2020,Credit Suisse,30.34,3,29,4
4,2020,JPMorgan Chase,34.09,7,22,5
5,2019,Credit Suisse,31.75,4,21,6
6,2019,BCV,29.70,2,16,7
7,2019,JPMorgan Chase,33.52,6,10,8
8,2020,Goldman Sachs,35.95,8,8,9
9,2019,Goldman Sachs,32.01,5,2,10


For the analysis this table, we kindly refer the reader of this notebook to the accompanying README file, which can be found on the GitHub repository.

In [ ]:
# Store the table
df_sorted.to_csv('industry_ranking')